In [40]:
from pymongo import MongoClient
import folium
import numpy as np
import pandas as pd

In [41]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [42]:
db, coll = connectCollection('companies','companies')

In [43]:
def getLocation(comp,i):
    latitude = comp['offices'][i]['latitude']
    longitude = comp['offices'][i]['longitude']
    loc = {
        'type':'Point',
        'coordinates':[longitude, latitude]
    }
    return loc

In [44]:
def getName(comp,i):
    name = comp['name']
    return name

I am going to filter all companies that have been founded after 2003, that have raised at least 1M dollars and that have a category code similar to a gaming company.

In [45]:
pipeline = [
    { "$unwind": "$offices"},
    {"$match":{ "$and": [  {"founded_year":{"$gt":2003}},
            {"funding_rounds.raised_amount":{"$gt":1000000}}, { "$or":[{"category_code":"web"},
            {"category_code":"software"},{"category_code":"games_video"},
            {"category_code":"hardware"},{"category_code":"mobile"},{"category_code":"music"},
            {"category_code":"photo_video"},{"category_code":"design"}]}] }}

    ]
results = list(coll.aggregate(pipeline))

In [46]:
name = []
city = []
country_code = []
longitude = []
latitude = []

for e in results:
    name.append(e["name"])
    city.append(e['offices']["city"])
    country_code.append(e['offices']["country_code"])
    for c in e["offices"].items():
        if c[0]=="longitude":
            longitude.append(c[1])
        elif c[0]=="latitude":
            latitude.append(c[1])

In [47]:
print(len(name),len(city),len(country_code),len(longitude),len(latitude))

1211 1211 1211 1211 1211


In [48]:
myData = {"name":name, "city":city, "country":country_code, "latitude":latitude, "longitude":longitude}

In [49]:
startup_df = pd.DataFrame(data=myData)

In [50]:
startup_df

,name,city,country,latitude,longitude
0,Wetpaint,Seattle,USA,47.603122,-122.333253
1,Wetpaint,New York,USA,40.723731,-73.996431
2,Geni,West Hollywood,USA,34.090368,-118.393064
3,Slacker,San Diego,USA,33.022176,-117.081406
4,Joost,New York,USA,40.746497,-74.009447
5,Babelgum,London,GBR,53.344104,-6.267494
6,Mahalo,Culver City,USA,34.017606,-118.487267
7,Kyte,San Francisco,USA,37.788482,-122.409173
8,Veoh,San Diego,USA,32.902266,-117.208340
9,Wesabe,San Francisco,USA,37.793148,-122.402567


Here I could use geocoding to obtain the null coordinates with the address of each offices, but for simplifying purposes, I am going to get rid of them.

In [51]:
startup_clean = startup_df.dropna()

In [52]:
startup_clean

,name,city,country,latitude,longitude
0,Wetpaint,Seattle,USA,47.603122,-122.333253
1,Wetpaint,New York,USA,40.723731,-73.996431
2,Geni,West Hollywood,USA,34.090368,-118.393064
3,Slacker,San Diego,USA,33.022176,-117.081406
4,Joost,New York,USA,40.746497,-74.009447
5,Babelgum,London,GBR,53.344104,-6.267494
6,Mahalo,Culver City,USA,34.017606,-118.487267
7,Kyte,San Francisco,USA,37.788482,-122.409173
8,Veoh,San Diego,USA,32.902266,-117.208340
9,Wesabe,San Francisco,USA,37.793148,-122.402567


In [53]:
startup_clean.index = pd.RangeIndex(len(startup_clean.index))

In [54]:
startup_clean.head(30)

,name,city,country,latitude,longitude
0,Wetpaint,Seattle,USA,47.603122,-122.333253
1,Wetpaint,New York,USA,40.723731,-73.996431
2,Geni,West Hollywood,USA,34.090368,-118.393064
3,Slacker,San Diego,USA,33.022176,-117.081406
4,Joost,New York,USA,40.746497,-74.009447
5,Babelgum,London,GBR,53.344104,-6.267494
6,Mahalo,Culver City,USA,34.017606,-118.487267
7,Kyte,San Francisco,USA,37.788482,-122.409173
8,Veoh,San Diego,USA,32.902266,-117.208340
9,Wesabe,San Francisco,USA,37.793148,-122.402567


In [236]:
startup_clean.to_csv(r'./input/startups_filtered_mongo.csv')

## APIs

In [58]:
import os
from dotenv import load_dotenv
import requests
load_dotenv()


True

In [59]:
def googleRequestAuthorized(lat,lon,r,keyword):

    # Function
    authToken = os.getenv("GOOGLE_API_TOKEN")
    if not authToken:
        raise ValueError("NECESITAS UN TOKEN")
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={},{}&radius={}&keyword={}&key={}".format(lat,lon,r,keyword,authToken)

    res = requests.get(url)
    data = res.json()
    return data

In [47]:
starbucks = googleRequestAuthorized("40.723731","-73.996431","200","starbucks")

In [23]:
vegan = googleRequestAuthorized("40.723731","-73.996431","200","vegan+restaurant")

In [49]:
club = googleRequestAuthorized("40.723731","-73.996431","500","club")

In [50]:
airport = googleRequestAuthorized("40.723731","-73.996431","30000","airport")

In [106]:
kindergarten = googleRequestAuthorized("40.723731","-73.996431","200","daycare")

In [24]:
l = vegan['results']

In [25]:
print(l[0]['name'])
print(l[0]['geometry']['location']['lat'])
print(l[0]['geometry']['location']['lng'])

by CHLOE.
40.7227818
-73.9972192


## There is a Starbucks in a radius of 100 meters

In [60]:
for i in range(len(startup_clean)) : 
    lat = startup_clean.loc[i, "latitude"]
    lon = startup_clean.loc[i, "longitude"]
    starbucks = googleRequestAuthorized(str(lat),str(lon),"100","starbucks")
    try:
        startup_clean.loc[i, 'starbucks'] = starbucks['results'][0]['name']
        startup_clean.loc[i, 'starbucks_lat'] = starbucks['results'][0]['geometry']['location']['lat']
        startup_clean.loc[i, 'starbucks_lon'] = starbucks['results'][0]['geometry']['location']['lng']

    except:
        startup_clean.loc[i, 'starbucks'] = np.nan
        startup_clean.loc[i, 'starbucks_lat'] = np.nan
        startup_clean.loc[i, 'starbucks_lon'] = np.nan

In [61]:
pd.set_option('display.max_rows', None)
display(startup_clean)

,name,city,country,latitude,longitude,starbucks,starbucks_lat,starbucks_lon
0,Wetpaint,Seattle,USA,47.603122,-122.333253,Starbucks,47.604156,-122.330827
1,Wetpaint,New York,USA,40.723731,-73.996431,Starbucks,40.722553,-73.997942
2,Geni,West Hollywood,USA,34.090368,-118.393064,NaN,NaN,NaN
3,Slacker,San Diego,USA,33.022176,-117.081406,Starbucks,33.023491,-117.081837
4,Joost,New York,USA,40.746497,-74.009447,NaN,NaN,NaN
5,Babelgum,London,GBR,53.344104,-6.267494,Starbucks Crampton Quay,53.346055,-6.262395
6,Mahalo,Culver City,USA,34.017606,-118.487267,Starbucks,34.017240,-118.489752
7,Kyte,San Francisco,USA,37.788482,-122.409173,Starbucks,37.789146,-122.408456
8,Veoh,San Diego,USA,32.902266,-117.208340,NaN,NaN,NaN
9,Wesabe,San Francisco,USA,37.793148,-122.402567,Starbucks,37.792183,-122.403770


In [88]:
startup_clean.to_csv(r'./input/starbucks_with_nan.csv')

In [62]:
starbucks_clean = startup_clean.dropna()

In [63]:
starbucks_clean.index = pd.RangeIndex(len(starbucks_clean.index))

In [67]:
pd.set_option('display.max_rows', None)
display(starbucks_clean)

,name,city,country,latitude,longitude,starbucks,starbucks_lat,starbucks_lon
0,Wetpaint,Seattle,USA,47.603122,-122.333253,Starbucks,47.604156,-122.330827
1,Wetpaint,New York,USA,40.723731,-73.996431,Starbucks,40.722553,-73.997942
2,Slacker,San Diego,USA,33.022176,-117.081406,Starbucks,33.023491,-117.081837
3,Babelgum,London,GBR,53.344104,-6.267494,Starbucks Crampton Quay,53.346055,-6.262395
4,Mahalo,Culver City,USA,34.017606,-118.487267,Starbucks,34.017240,-118.489752
5,Kyte,San Francisco,USA,37.788482,-122.409173,Starbucks,37.789146,-122.408456
6,Wesabe,San Francisco,USA,37.793148,-122.402567,Starbucks,37.792183,-122.403770
7,AdaptiveBlue,NYC,USA,40.801358,-74.337200,Starbucks,40.798594,-74.340772
8,Pando Networks,New York,USA,40.722655,-73.998730,Starbucks,40.722553,-73.997942
9,Livestream,New York,USA,40.726155,-73.995625,Starbucks,40.727212,-73.995485


In [66]:
starbucks_clean.to_csv(r'./input/starbucks_without_nan.csv')

## There is an airport in a radius of 20 km.

In [68]:
for i in range(len(starbucks_clean)) : 
    lat = starbucks_clean.loc[i, "latitude"]
    lon = starbucks_clean.loc[i, "longitude"]
    airport = googleRequestAuthorized(str(lat),str(lon),"20000","airport")
    try:
        starbucks_clean.loc[i, 'airport'] = airport['results'][0]['name']
        starbucks_clean.loc[i, 'air_lat'] = airport['results'][0]['geometry']['location']['lat']
        starbucks_clean.loc[i, 'air_lon'] = airport['results'][0]['geometry']['location']['lng']

    except:
        starbucks_clean.loc[i, 'airport'] = np.nan
        starbucks_clean.loc[i, 'air_lat'] = np.nan
        starbucks_clean.loc[i, 'air_lon'] = np.nan

In [72]:
pd.set_option('display.max_rows', None)
display(airports_clean)

,name,city,country,latitude,longitude,starbucks,starbucks_lat,starbucks_lon,airport,air_lat,air_lon
0,Wetpaint,Seattle,USA,47.603122,-122.333253,Starbucks,47.604156,-122.330827,Seattle-Tacoma International Airport,47.450250,-122.308817
1,Wetpaint,New York,USA,40.723731,-73.996431,Starbucks,40.722553,-73.997942,John F. Kennedy International Airport,40.641311,-73.778139
2,Slacker,San Diego,USA,33.022176,-117.081406,Starbucks,33.023491,-117.081837,McClellan-Palomar Airport,33.126822,-117.279241
3,Babelgum,London,GBR,53.344104,-6.267494,Starbucks Crampton Quay,53.346055,-6.262395,Dublin Airport,53.426448,-6.249910
4,Mahalo,Culver City,USA,34.017606,-118.487267,Starbucks,34.017240,-118.489752,Los Angeles International Airport,33.941589,-118.408530
5,Kyte,San Francisco,USA,37.788482,-122.409173,Starbucks,37.789146,-122.408456,San Francisco International Airport,37.621313,-122.378955
6,Wesabe,San Francisco,USA,37.793148,-122.402567,Starbucks,37.792183,-122.403770,San Francisco International Airport,37.621313,-122.378955
7,AdaptiveBlue,NYC,USA,40.801358,-74.337200,Starbucks,40.798594,-74.340772,Newark Liberty International Airport,40.689531,-74.174462
8,Pando Networks,New York,USA,40.722655,-73.998730,Starbucks,40.722553,-73.997942,John F. Kennedy International Airport,40.641311,-73.778139
9,Livestream,New York,USA,40.726155,-73.995625,Starbucks,40.727212,-73.995485,John F. Kennedy International Airport,40.641311,-73.778139


In [70]:
airports_clean = starbucks_clean.dropna()

In [71]:
airports_clean.index = pd.RangeIndex(len(airports_clean.index))

In [73]:
airports_clean.to_csv(r'./input/airports_without_nan.csv')

## There is a Day Care Center in a radius of 100 meters

In [74]:
def add_places(df,place,radius):
    for i in range(len(df)) : 
        lat = df.loc[i, "latitude"]
        lon = df.loc[i, "longitude"]
        answer = googleRequestAuthorized(str(lat),str(lon),str(radius),str(place))
        try:
            df.loc[i, str(place)] = answer['results'][0]['name']
            df.loc[i, str(place)+'_lat'] = answer['results'][0]['geometry']['location']['lat']
            df.loc[i, str(place)+'_lon'] = answer['results'][0]['geometry']['location']['lng']

        except:
            df.loc[i, str(place)] = np.nan
            df.loc[i, str(place)+'_lat'] = np.nan
            df.loc[i, str(place)+'_lon'] = np.nan

In [75]:
add_places(airports_clean,"daycare","100")

In [76]:
airports_clean

,name,city,country,latitude,longitude,starbucks,starbucks_lat,starbucks_lon,airport,air_lat,air_lon,daycare,daycare_lat,daycare_lon
0,Wetpaint,Seattle,USA,47.603122,-122.333253,Starbucks,47.604156,-122.330827,Seattle-Tacoma International Airport,47.450250,-122.308817,Cosmopolitan Kids Downtown Academy,47.605686,-122.334997
1,Wetpaint,New York,USA,40.723731,-73.996431,Starbucks,40.722553,-73.997942,John F. Kennedy International Airport,40.641311,-73.778139,NaN,NaN,NaN
2,Slacker,San Diego,USA,33.022176,-117.081406,Starbucks,33.023491,-117.081837,McClellan-Palomar Airport,33.126822,-117.279241,NaN,NaN,NaN
3,Babelgum,London,GBR,53.344104,-6.267494,Starbucks Crampton Quay,53.346055,-6.262395,Dublin Airport,53.426448,-6.249910,NaN,NaN,NaN
4,Mahalo,Culver City,USA,34.017606,-118.487267,Starbucks,34.017240,-118.489752,Los Angeles International Airport,33.941589,-118.408530,NaN,NaN,NaN
5,Kyte,San Francisco,USA,37.788482,-122.409173,Starbucks,37.789146,-122.408456,San Francisco International Airport,37.621313,-122.378955,NaN,NaN,NaN
6,Wesabe,San Francisco,USA,37.793148,-122.402567,Starbucks,37.792183,-122.403770,San Francisco International Airport,37.621313,-122.378955,Bright Horizons,37.792111,-122.403927
7,AdaptiveBlue,NYC,USA,40.801358,-74.337200,Starbucks,40.798594,-74.340772,Newark Liberty International Airport,40.689531,-74.174462,NaN,NaN,NaN
8,Pando Networks,New York,USA,40.722655,-73.998730,Starbucks,40.722553,-73.997942,John F. Kennedy International Airport,40.641311,-73.778139,SoHo Child Care,40.723374,-74.001060
9,Livestream,New York,USA,40.726155,-73.995625,Starbucks,40.727212,-73.995485,John F. Kennedy International Airport,40.641311,-73.778139,Sompit Child Care,40.727806,-73.997972


In [77]:
daycare_clean = airports_clean.dropna()

In [78]:
daycare_clean.index = pd.RangeIndex(len(daycare_clean.index))

In [79]:
daycare_clean.to_csv(r'./input/daycare_without_nan.csv')

In [80]:
daycare_clean

,name,city,country,latitude,longitude,starbucks,starbucks_lat,starbucks_lon,airport,air_lat,air_lon,daycare,daycare_lat,daycare_lon
0,Wetpaint,Seattle,USA,47.603122,-122.333253,Starbucks,47.604156,-122.330827,Seattle-Tacoma International Airport,47.450250,-122.308817,Cosmopolitan Kids Downtown Academy,47.605686,-122.334997
1,Wesabe,San Francisco,USA,37.793148,-122.402567,Starbucks,37.792183,-122.403770,San Francisco International Airport,37.621313,-122.378955,Bright Horizons,37.792111,-122.403927
2,Pando Networks,New York,USA,40.722655,-73.998730,Starbucks,40.722553,-73.997942,John F. Kennedy International Airport,40.641311,-73.778139,SoHo Child Care,40.723374,-74.001060
3,Livestream,New York,USA,40.726155,-73.995625,Starbucks,40.727212,-73.995485,John F. Kennedy International Airport,40.641311,-73.778139,Sompit Child Care,40.727806,-73.997972
4,Netvibes,Paris,FRA,48.870806,2.346680,Starbucks,48.870675,2.346912,Paris-Charles De Gaulle,49.009691,2.547925,Municipal Collective Nursery - Thorel,48.869689,2.349065
5,FeedBurner,Chicago,USA,41.889474,-87.628912,Starbucks,41.890160,-87.631251,Chicago Midway International Airport,41.786776,-87.752188,Nannies of Children's Learning Place,41.889873,-87.627482
6,TVtrip,Paris,FRA,48.856667,2.350987,Starbucks,48.856820,2.354893,Paris-Charles De Gaulle,49.009691,2.547925,Day Care Municipal - Lobau,48.856031,2.352951
7,Bebo,San Francisco,USA,37.782103,-122.401116,Starbucks,37.784833,-122.399960,San Francisco International Airport,37.621313,-122.378955,Modern Education Family Childcare - Yerba Buena,37.781321,-122.401216
8,Loomia,San Francisco,USA,37.796396,-122.404869,Starbucks,37.794632,-122.402728,San Francisco International Airport,37.621313,-122.378955,Wu Yee Children's Services - Lok Yuen CDC & Jo...,37.794183,-122.407689
9,Terabitz,Palo Alto,USA,37.437328,-122.159928,Peet's Coffee,37.438064,-122.159322,San Carlos Airport,37.515375,-122.250305,Odyssey Montessori,37.439599,-122.157198


## There is a nightclub in less than 150 meters

In [81]:
add_places(daycare_clean,"nightclub","150")

In [82]:
daycare_clean

,name,city,country,latitude,longitude,starbucks,starbucks_lat,starbucks_lon,airport,air_lat,air_lon,daycare,daycare_lat,daycare_lon,nightclub,nightclub_lat,nightclub_lon
0,Wetpaint,Seattle,USA,47.603122,-122.333253,Starbucks,47.604156,-122.330827,Seattle-Tacoma International Airport,47.450250,-122.308817,Cosmopolitan Kids Downtown Academy,47.605686,-122.334997,Trinity,47.601513,-122.333296
1,Wesabe,San Francisco,USA,37.793148,-122.402567,Starbucks,37.792183,-122.403770,San Francisco International Airport,37.621313,-122.378955,Bright Horizons,37.792111,-122.403927,Club Blend,37.791944,-122.404517
2,Pando Networks,New York,USA,40.722655,-73.998730,Starbucks,40.722553,-73.997942,John F. Kennedy International Airport,40.641311,-73.778139,SoHo Child Care,40.723374,-74.001060,GoldBar,40.720375,-73.997118
3,Livestream,New York,USA,40.726155,-73.995625,Starbucks,40.727212,-73.995485,John F. Kennedy International Airport,40.641311,-73.778139,Sompit Child Care,40.727806,-73.997972,House of Hunter,40.724738,-73.997656
4,Netvibes,Paris,FRA,48.870806,2.346680,Starbucks,48.870675,2.346912,Paris-Charles De Gaulle,49.009691,2.547925,Municipal Collective Nursery - Thorel,48.869689,2.349065,S.i.r.,48.870445,2.347435
5,FeedBurner,Chicago,USA,41.889474,-87.628912,Starbucks,41.890160,-87.631251,Chicago Midway International Airport,41.786776,-87.752188,Nannies of Children's Learning Place,41.889873,-87.627482,Underground,41.890952,-87.630203
6,TVtrip,Paris,FRA,48.856667,2.350987,Starbucks,48.856820,2.354893,Paris-Charles De Gaulle,49.009691,2.547925,Day Care Municipal - Lobau,48.856031,2.352951,Le Klub,48.859106,2.347902
7,Bebo,San Francisco,USA,37.782103,-122.401116,Starbucks,37.784833,-122.399960,San Francisco International Airport,37.621313,-122.378955,Modern Education Family Childcare - Yerba Buena,37.781321,-122.401216,The Grand,37.779100,-122.397900
8,Loomia,San Francisco,USA,37.796396,-122.404869,Starbucks,37.794632,-122.402728,San Francisco International Airport,37.621313,-122.378955,Wu Yee Children's Services - Lok Yuen CDC & Jo...,37.794183,-122.407689,HUE,37.797948,-122.404785
9,Terabitz,Palo Alto,USA,37.437328,-122.159928,Peet's Coffee,37.438064,-122.159322,San Carlos Airport,37.515375,-122.250305,Odyssey Montessori,37.439599,-122.157198,NaN,NaN,NaN


In [83]:
nightclub_clean = daycare_clean.dropna()

In [84]:
nightclub_clean.index = pd.RangeIndex(len(nightclub_clean.index))

In [85]:
nightclub_clean

,name,city,country,latitude,longitude,starbucks,starbucks_lat,starbucks_lon,airport,air_lat,air_lon,daycare,daycare_lat,daycare_lon,nightclub,nightclub_lat,nightclub_lon
0,Wetpaint,Seattle,USA,47.603122,-122.333253,Starbucks,47.604156,-122.330827,Seattle-Tacoma International Airport,47.450250,-122.308817,Cosmopolitan Kids Downtown Academy,47.605686,-122.334997,Trinity,47.601513,-122.333296
1,Wesabe,San Francisco,USA,37.793148,-122.402567,Starbucks,37.792183,-122.403770,San Francisco International Airport,37.621313,-122.378955,Bright Horizons,37.792111,-122.403927,Club Blend,37.791944,-122.404517
2,Pando Networks,New York,USA,40.722655,-73.998730,Starbucks,40.722553,-73.997942,John F. Kennedy International Airport,40.641311,-73.778139,SoHo Child Care,40.723374,-74.001060,GoldBar,40.720375,-73.997118
3,Livestream,New York,USA,40.726155,-73.995625,Starbucks,40.727212,-73.995485,John F. Kennedy International Airport,40.641311,-73.778139,Sompit Child Care,40.727806,-73.997972,House of Hunter,40.724738,-73.997656
4,Netvibes,Paris,FRA,48.870806,2.346680,Starbucks,48.870675,2.346912,Paris-Charles De Gaulle,49.009691,2.547925,Municipal Collective Nursery - Thorel,48.869689,2.349065,S.i.r.,48.870445,2.347435
5,FeedBurner,Chicago,USA,41.889474,-87.628912,Starbucks,41.890160,-87.631251,Chicago Midway International Airport,41.786776,-87.752188,Nannies of Children's Learning Place,41.889873,-87.627482,Underground,41.890952,-87.630203
6,TVtrip,Paris,FRA,48.856667,2.350987,Starbucks,48.856820,2.354893,Paris-Charles De Gaulle,49.009691,2.547925,Day Care Municipal - Lobau,48.856031,2.352951,Le Klub,48.859106,2.347902
7,Bebo,San Francisco,USA,37.782103,-122.401116,Starbucks,37.784833,-122.399960,San Francisco International Airport,37.621313,-122.378955,Modern Education Family Childcare - Yerba Buena,37.781321,-122.401216,The Grand,37.779100,-122.397900
8,Loomia,San Francisco,USA,37.796396,-122.404869,Starbucks,37.794632,-122.402728,San Francisco International Airport,37.621313,-122.378955,Wu Yee Children's Services - Lok Yuen CDC & Jo...,37.794183,-122.407689,HUE,37.797948,-122.404785
9,Eventbrite,San Francisco,USA,37.779507,-122.390710,Starbucks,37.777035,-122.393641,San Francisco International Airport,37.621313,-122.378955,Wonderful Children Day Care,37.782291,-122.388807,LL World HQ,37.782420,-122.395113


In [86]:
nightclub_clean.to_csv(r'./input/nightclub_without_nan.csv')

## There is a vegan restaurant in a radius of 50 meters

In [87]:
add_places(nightclub_clean,"vegan+restaurant","50")

In [88]:
nightclub_clean

,name,city,country,latitude,longitude,starbucks,starbucks_lat,starbucks_lon,airport,air_lat,air_lon,daycare,daycare_lat,daycare_lon,nightclub,nightclub_lat,nightclub_lon,vegan+restaurant,vegan+restaurant_lat,vegan+restaurant_lon
0,Wetpaint,Seattle,USA,47.603122,-122.333253,Starbucks,47.604156,-122.330827,Seattle-Tacoma International Airport,47.450250,-122.308817,Cosmopolitan Kids Downtown Academy,47.605686,-122.334997,Trinity,47.601513,-122.333296,Cafe Zum Zum,47.604655,-122.333455
1,Wesabe,San Francisco,USA,37.793148,-122.402567,Starbucks,37.792183,-122.403770,San Francisco International Airport,37.621313,-122.378955,Bright Horizons,37.792111,-122.403927,Club Blend,37.791944,-122.404517,Enjoy Vegetarian,37.795818,-122.405248
2,Pando Networks,New York,USA,40.722655,-73.998730,Starbucks,40.722553,-73.997942,John F. Kennedy International Airport,40.641311,-73.778139,SoHo Child Care,40.723374,-74.001060,GoldBar,40.720375,-73.997118,Le Botaniste,40.720493,-74.000229
3,Livestream,New York,USA,40.726155,-73.995625,Starbucks,40.727212,-73.995485,John F. Kennedy International Airport,40.641311,-73.778139,Sompit Child Care,40.727806,-73.997972,House of Hunter,40.724738,-73.997656,Siggy's Good Food,40.724751,-73.993124
4,Netvibes,Paris,FRA,48.870806,2.346680,Starbucks,48.870675,2.346912,Paris-Charles De Gaulle,49.009691,2.547925,Municipal Collective Nursery - Thorel,48.869689,2.349065,S.i.r.,48.870445,2.347435,Vietnam In Paris,48.868414,2.347870
5,FeedBurner,Chicago,USA,41.889474,-87.628912,Starbucks,41.890160,-87.631251,Chicago Midway International Airport,41.786776,-87.752188,Nannies of Children's Learning Place,41.889873,-87.627482,Underground,41.890952,-87.630203,LYFE Kitchen,41.889843,-87.630847
6,TVtrip,Paris,FRA,48.856667,2.350987,Starbucks,48.856820,2.354893,Paris-Charles De Gaulle,49.009691,2.547925,Day Care Municipal - Lobau,48.856031,2.352951,Le Klub,48.859106,2.347902,Wild & The Moon @ Lafayette Anticipations,48.859183,2.354776
7,Bebo,San Francisco,USA,37.782103,-122.401116,Starbucks,37.784833,-122.399960,San Francisco International Airport,37.621313,-122.378955,Modern Education Family Childcare - Yerba Buena,37.781321,-122.401216,The Grand,37.779100,-122.397900,NaN,NaN,NaN
8,Loomia,San Francisco,USA,37.796396,-122.404869,Starbucks,37.794632,-122.402728,San Francisco International Airport,37.621313,-122.378955,Wu Yee Children's Services - Lok Yuen CDC & Jo...,37.794183,-122.407689,HUE,37.797948,-122.404785,Enjoy Vegetarian,37.795818,-122.405248
9,Eventbrite,San Francisco,USA,37.779507,-122.390710,Starbucks,37.777035,-122.393641,San Francisco International Airport,37.621313,-122.378955,Wonderful Children Day Care,37.782291,-122.388807,LL World HQ,37.782420,-122.395113,SAJJ Mediterranean SoMa,37.781306,-122.391719


In [89]:
all_clean = nightclub_clean.dropna()

In [90]:
all_clean.index = pd.RangeIndex(len(all_clean.index))

In [235]:
all_clean.to_csv(r'./input/all_clean.csv')

In [136]:
all_clean

,tech_startups,city,country,latitude,longitude,starbucks,starbucks_lat,starbucks_lon,airport,air_lat,air_lon,daycare,daycare_lat,daycare_lon,nightclub,nightclub_lat,nightclub_lon,vegan_restaurant,vegan_restaurant_lat,vegan_restaurant_lon
0,Wetpaint,Seattle,USA,47.603122,-122.333253,Starbucks,47.604156,-122.330827,Seattle-Tacoma International Airport,47.450250,-122.308817,Cosmopolitan Kids Downtown Academy,47.605686,-122.334997,Trinity,47.601513,-122.333296,Cafe Zum Zum,47.604655,-122.333455
1,Wesabe,San Francisco,USA,37.793148,-122.402567,Starbucks,37.792183,-122.403770,San Francisco International Airport,37.621313,-122.378955,Bright Horizons,37.792111,-122.403927,Club Blend,37.791944,-122.404517,Enjoy Vegetarian,37.795818,-122.405248
2,Pando Networks,New York,USA,40.722655,-73.998730,Starbucks,40.722553,-73.997942,John F. Kennedy International Airport,40.641311,-73.778139,SoHo Child Care,40.723374,-74.001060,GoldBar,40.720375,-73.997118,Le Botaniste,40.720493,-74.000229
3,Livestream,New York,USA,40.726155,-73.995625,Starbucks,40.727212,-73.995485,John F. Kennedy International Airport,40.641311,-73.778139,Sompit Child Care,40.727806,-73.997972,House of Hunter,40.724738,-73.997656,Siggy's Good Food,40.724751,-73.993124
4,Netvibes,Paris,FRA,48.870806,2.346680,Starbucks,48.870675,2.346912,Paris-Charles De Gaulle,49.009691,2.547925,Municipal Collective Nursery - Thorel,48.869689,2.349065,S.i.r.,48.870445,2.347435,Vietnam In Paris,48.868414,2.347870
5,FeedBurner,Chicago,USA,41.889474,-87.628912,Starbucks,41.890160,-87.631251,Chicago Midway International Airport,41.786776,-87.752188,Nannies of Children's Learning Place,41.889873,-87.627482,Underground,41.890952,-87.630203,LYFE Kitchen,41.889843,-87.630847
6,TVtrip,Paris,FRA,48.856667,2.350987,Starbucks,48.856820,2.354893,Paris-Charles De Gaulle,49.009691,2.547925,Day Care Municipal - Lobau,48.856031,2.352951,Le Klub,48.859106,2.347902,Wild & The Moon @ Lafayette Anticipations,48.859183,2.354776
7,Loomia,San Francisco,USA,37.796396,-122.404869,Starbucks,37.794632,-122.402728,San Francisco International Airport,37.621313,-122.378955,Wu Yee Children's Services - Lok Yuen CDC & Jo...,37.794183,-122.407689,HUE,37.797948,-122.404785,Enjoy Vegetarian,37.795818,-122.405248
8,Eventbrite,San Francisco,USA,37.779507,-122.390710,Starbucks,37.777035,-122.393641,San Francisco International Airport,37.621313,-122.378955,Wonderful Children Day Care,37.782291,-122.388807,LL World HQ,37.782420,-122.395113,SAJJ Mediterranean SoMa,37.781306,-122.391719
9,Seesmic,San Francisco,USA,37.775196,-122.419204,Starbucks,37.777060,-122.417347,San Francisco International Airport,37.621313,-122.378955,Marin Day Schools Market Street,37.777204,-122.417349,"THEX RELIGIOÚZ ROMEX, ITALIAX!",37.774929,-122.419415,Ananda Fuara,37.777874,-122.416292


In [107]:
all_clean = all_clean.rename(columns={"vegan+restaurant": "vegan_restaurant", "vegan+restaurant_lat": "vegan_restaurant_lat", "vegan+restaurant_lon": "vegan_restaurant_lon"})

In [114]:
all_clean = all_clean.rename(columns={"name": "tech_startups"})

In [131]:
all_clean['city'] = all_clean['city'].replace({'Copenhagen V':'Copenhagen', 'Montreal, Quebec':'Montreal', 'MontrÃ©al, Qc':'Montreal'})


In [132]:
city_count = all_clean.groupby(['city']).count()


In [216]:
city_sorted = city_count.sort_values(['tech_startups'], ascending=[False]).head(50)


In [217]:
city_sorted['tech_startups']

city
New York               52
San Francisco          52
Seattle                10
Boston                  6
Paris                   5
Chicago                 5
Toronto                 5
Palo Alto               4
Los Angeles             4
Mountain View           4
Montreal                3
Boulder                 3
Bellevue                3
London                  3
Amsterdam               2
Reston                  2
Redwood City            2
Menlo Park              2
Corte Madera            2
Copenhagen              2
Sydney                  2
Berlin                  2
San Jose                1
San Mateo               1
Santa Clara             1
Singapore               1
Stockholm               1
Sunnyvale               1
Pittsburgh              1
Santa Monica            1
                        1
Olpe                    1
New Delhi               1
Natick                  1
Melbourne, Victoria     1
Lancaster               1
Honolulu                1
Halifax                 1
Greenbe

In [232]:
city_df = all_clean.loc[all_clean.city == 'Chicago']

In [233]:
lati = city_df['latitude'].iloc[0]
long = city_df['longitude'].iloc[0]

map_city=folium.Map(location=[float(lati),float(long)], zoom_start=12)

for index, row in city_df.iterrows():
    folium.Marker((row['latitude'],row['longitude']),
                    radius=2,
                    icon=folium.Icon(icon='home',color='red'),
                    tooltip='Cool Tech Startup: '+str(row['tech_startups'])
                   ).add_to(map_city)
    
for index, row in city_df.iterrows():
    folium.Marker((row['starbucks_lat'],row['starbucks_lon']),
                    radius=2,
                    icon=folium.Icon(icon='leaf',color='green'),
                    tooltip=str(row['starbucks'])
                   ).add_to(map_city)
    
for index, row in city_df.iterrows():
    folium.Marker((row['air_lat'],row['air_lon']),
                    radius=2,
                    icon=folium.Icon(icon='plane',color='lightgray'),
                    tooltip=str(row['airport'])
                   ).add_to(map_city)
    
    
for index, row in city_df.iterrows():
    folium.Marker((row['daycare_lat'],row['daycare_lon']),
                    radius=2,
                    icon=folium.Icon(icon='heart',color='blue'),
                    tooltip='Kindergarten: '+str(row['daycare'])
                   ).add_to(map_city)
    
    
for index, row in city_df.iterrows():
    folium.Marker((row['nightclub_lat'],row['nightclub_lon']),
                    radius=2,
                    icon=folium.Icon(icon='glass',color='darkpurple'),
                    tooltip='Nightclub: '+str(row['nightclub'])
                   ).add_to(map_city)
    
for index, row in city_df.iterrows():
    folium.Marker((row['vegan_restaurant_lat'],row['vegan_restaurant_lon']),
                    radius=2,
                    icon=folium.Icon(icon='cutlery',color='darkgreen'),
                    tooltip='Vegan: '+str(row['vegan_restaurant'])
                   ).add_to(map_city)

In [234]:
map_city